# Introduction
In this file, whole brain MVPA is applied on Z-stat files from first level analysis I have done on naive space, ART applied images, with each stimuli giving one Z-stat image.

MPVA is applied for all voxels of the brain. For each participant, data is split into the number of runs, with data from one run is used for test, and the rest is used for training. 

## Future works
Testing on various types of classifers, wokring on classifier performance are works to be done.
    

In [6]:
import os
import glob
import numpy as np
import pandas as pd
import os.path as op
import warnings
import matplotlib.pyplot as plt
plt.style.use('classic')
warnings.filterwarnings("ignore")
from scipy.io import loadmat
from glob import glob
import nibabel as nib
import math
import itertools
import timeit
from timeit import default_timer as timer

%matplotlib inline

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
#from sklearn.svm import LinearSVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

In [8]:
subj_list_file='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/subj_simultaneous.txt'
subj_list=list()
f = open(subj_list_file, 'r')
line=f.readline()
while line:
    subj_list.append(line[0:-1])
    oldline=line
    line = f.readline()
subj_list.pop()
subj_list.append(oldline)
f.close()
Nsubj=len(subj_list)

In [9]:
# note: I don't know why the last item is repeated! but it doesn't make a misatke

conditions=['Neu','Neg','Pos','Targ']
ZstatInfo=pd.DataFrame(columns=['Subj','Run','Folder']+conditions)
for subj in subj_list:
    #print('Participant: '+subj)
    # Zstat files and their condisitons are in SPM filesWe repeat loading in the dataframe to avoid dividing the onsets by 2 multiple times ...
    SubjSPMFolder='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/SPM/'+subj+'/3cond_dist2/'
    os.path.isdir(SubjSPMFolder)
    myZstat=[]
    
    nRuns = len(os.listdir(SubjSPMFolder))
    iRuns = 0
    for myRun in os.listdir(SubjSPMFolder):
        ZstatInfotmp=pd.DataFrame(columns=ZstatInfo.columns,index=[0])
        ZstatInfotmp['Subj'][0]=subj
        
        RunSubjSPM=SubjSPMFolder+myRun+'/SPM.mat'
        ZstatInfotmp['Folder'][0]=SubjSPMFolder+myRun+'/'

        #print('   loading SPM.mat for '+myRun+'...')
        mat = loadmat(RunSubjSPM)
        mdata=mat['SPM'][0,0]
        ndata=mdata['Sess'][0,0]
        odata=ndata['U']
        pdata=odata['name'][0]
        ZstatNames=[pdata[i][0,0][0] for i in range(len(pdata))]
        myZstat.append(ZstatNames)

        ZstatInfotmp['Run'][0]=iRuns+1
        old_ind=len(myZstat[0])+1
        for iCond in reversed(conditions):    
            Start_ind=myZstat[0].index(iCond+'_1')+1
            #print(Start_ind)
            ZstatInfotmp[iCond][0]=['spmT_'+str(val).zfill(4)+'.nii' for val in list(range(Start_ind, old_ind))]#list(range(Start_ind, old_ind))
            old_ind=Start_ind
        ZstatInfo=ZstatInfo.append(ZstatInfotmp,ignore_index=True)
        iRuns = iRuns+1
 

In [10]:
# find Voxel dimension by openning a file
MyZstat=ZstatInfo.loc[(ZstatInfo['Subj']==subj) & (ZstatInfo['Run']==1)]
MyZstat.reset_index(inplace = True, drop = True)
MyFile=MyZstat['Folder'][0]+MyZstat['Neu'][0][0]
print(MyFile)
data = nib.load(MyFile).get_data()
voxel_dims = data.shape
print(voxel_dims)


C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/SPM/TP_05162014/3cond_dist2/run01/spmT_0001.nii
(64, 64, 28)


In [33]:
ZstatInfo['Subj']

0      AB_04152014
1      AB_04152014
2      AB_04152014
3      AB_04152014
4      AB_04152014
          ...     
117    TP_05162014
118    TP_05162014
119    TP_05162014
120    TP_05162014
121    TP_05162014
Name: Subj, Length: 122, dtype: object

In [46]:
MyZstat=ZstatInfo.loc[(ZstatInfo['Subj']=='AB_04152014') & (ZstatInfo['Run']==5)]
MyZstat.reset_index(inplace = True, drop = True)
MyFile3=MyZstat['Folder'][0]+MyZstat['Neg'][0][4]
print(MyFile3)
hdr3=nib.load(MyFile3).header.structarr

C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/SPM/AB_04152014/3cond_dist2/run05/spmT_0011.nii


In [50]:
mydata=nib.load(MyFile3).get_data()
img_affine=nib.load(MyFile3).affine
nft_img = nib.Nifti1Image(mydata, img_affine)
nib.save(nft_img, 'C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/tmp.nii.gz')
#img = nib.Nifti1Image(nib.load(MyFile3).get_data(), hdr3)

In [40]:
hdr=nib.load(MyFile).header
hdr2=nib.load(MyFile2).header
hdr2.structarr

array((348, b'', b'', 0, 0, b'r', 0, [ 3, 64, 64, 28,  1,  1,  1,  1], 0., 0., 0., 0, 16, 32, 0, [-1.,  4.,  4.,  4.,  0.,  0.,  0.,  0.], 0., nan, nan, 0, 0, 10, 0., 0., 0., 0., 0, 0, b'SPM{T_[109.0]} - contrast 14: Neg_6', b'', 2, 2, -0.01092714, 0.9963955, -0.08073097, 134.997, -150.5094, -6.896915, [-3.9945722e+00, -7.1830869e-02, -1.9553673e-01,  1.3499699e+02], [-1.0237324e-01,  3.9469049e+00,  6.4145285e-01, -1.5050940e+02], [-0.18142223, -0.64558685,  3.9433873 , -6.896915  ], b'', b'n+1'),
      dtype=[('sizeof_hdr', '<i4'), ('data_type', 'S10'), ('db_name', 'S18'), ('extents', '<i4'), ('session_error', '<i2'), ('regular', 'S1'), ('dim_info', 'u1'), ('dim', '<i2', (8,)), ('intent_p1', '<f4'), ('intent_p2', '<f4'), ('intent_p3', '<f4'), ('intent_code', '<i2'), ('datatype', '<i2'), ('bitpix', '<i2'), ('slice_start', '<i2'), ('pixdim', '<f4', (8,)), ('vox_offset', '<f4'), ('scl_slope', '<f4'), ('scl_inter', '<f4'), ('slice_end', '<i2'), ('slice_code', 'u1'), ('xyzt_units', 'u1'),

In [31]:
hdr.structarr

array((348, b'', b'', 0, 0, b'r', 0, [ 3, 64, 64, 28,  1,  1,  1,  1], 0., 0., 0., 0, 16, 32, 0, [-1.,  4.,  4.,  4.,  0.,  0.,  0.,  0.], 0., nan, nan, 0, 0, 10, 0., 0., 0., 0., 0, 0, b'SPM{T_[114.0]} - contrast 1: Neu_1', b'', 2, 2, 0.00123516, 0.9880689, -0.15400483, 127.28155, -156.23947, -5.182937, [-3.99998140e+00,  1.08797550e-02, -5.64050674e-03,  1.27281555e+02], [ 8.6470842e-03,  3.8102479e+00,  1.2173480e+00, -1.5623947e+02], [-0.00868404, -1.2173302 ,  3.8102536 , -5.1829376 ], b'', b'n+1'),
      dtype=[('sizeof_hdr', '<i4'), ('data_type', 'S10'), ('db_name', 'S18'), ('extents', '<i4'), ('session_error', '<i2'), ('regular', 'S1'), ('dim_info', 'u1'), ('dim', '<i2', (8,)), ('intent_p1', '<f4'), ('intent_p2', '<f4'), ('intent_p3', '<f4'), ('intent_code', '<i2'), ('datatype', '<i2'), ('bitpix', '<i2'), ('slice_start', '<i2'), ('pixdim', '<f4', (8,)), ('vox_offset', '<f4'), ('scl_slope', '<f4'), ('scl_inter', '<f4'), ('slice_end', '<i2'), ('slice_code', 'u1'), ('xyzt_units', '

## notes on AUC calculation for multilabel case:
There are two options available: 
* multi_class='ovr'(One-vs-rest): Computes the AUC of each class against the rest, This treats the multiclass case in the same way as the multilabel case. 
 * Sensitive to class imbalance even when average == 'macro', because class imbalance affects the composition of each of the ‘rest’ groupings.
* multi_class='ovo'(One-vs-one): Computes the average AUC of all possible pairwise combinations of classes. 
 * Insensitive to class imbalance when average == 'macro'.
 * In the pairwise AUC calculation, if a sample is labeled wrongly as a another class (not in paired), it will be dropped from the calculation, resulting in a higher rate

Comparison results for 'AM_05082014':

* Accuracy: [0.71, 0.75,0.57, 0.54, 0.54]
* AUC, with multi_class='ovr':[0.83576389 0.89131944 0.84791667 0.83854167 0.78506944] 
* AUC, with multi_class='ovo': [0.81163194 0.875      0.81076389 0.7578125  0.72222222]
* AUC, with multi_class='ovo', average = 'macro':[0.80381944 0.875      0.81597222 0.81336806 0.71354167]

## Conclusion: 
Overall 'ovo' with average == 'macro' gives the lowest value, but still higher than accuracy calculations
We use AUC and Accuracy for the purpose of this analysis

In [14]:
# This code does classification based on whole brain volume
CLconditions=['Neu','Neg','Targ']

select100best = SelectKBest(f_classif, k=100)
##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipe = Pipeline([('scaler', StandardScaler()), ('ufs', select100best), ('clf', clf)])

#subj_list_one=[subj_list[1]]; 
Nsubj=len(subj_list)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(subj_list):
    X=np.empty((0, np.prod(voxel_dims)), int)
    Y=np.array([])    
    Gr=np.array([])    
    for iRun in ZstatInfo.loc[ZstatInfo['Subj']==subj]['Run']:
        MyZstat=ZstatInfo.loc[(ZstatInfo['Subj']==subj) & (ZstatInfo['Run']==iRun)]
        MyZstat.reset_index(inplace = True, drop = True)
        #MyZstat=MyZstat.loc[0]
        X_run=np.empty((0, np.prod(voxel_dims)), int)
        Y_run=np.array([])
        Gr_run=np.array([])
        for iCond,CondName in enumerate(CLconditions):
            #print(MyZstat[CondName][0])
            X_tmp = np.zeros((len(MyZstat[CondName][0]), np.prod(voxel_dims)))
            Y_tmp = np.zeros(len(MyZstat[CondName][0]))
            for MyInd,MyNii in enumerate(MyZstat[CondName][0]):
                MyFile=MyZstat['Folder'][0]+MyNii
                data = nib.load(MyFile).get_data()
                dataR = data.ravel()
                X_tmp[MyInd,:] = dataR
                Y_tmp[MyInd]=int(iCond)+1
                
            X_run=np.concatenate((X_run,X_tmp),axis = 0)
            Y_run=np.concatenate((Y_run,Y_tmp),axis = 0)
            Gr_run=np.concatenate((Gr_run,np.repeat(iRun,Y_tmp.shape)),axis = 0)
            #print(Y.shape)
        X=np.concatenate((X,X_run),axis = 0)
        Y=np.concatenate((Y,Y_run),axis = 0)
        Gr=np.concatenate((Gr,Gr_run),axis = 0)

    # print(Gr)
    gkf = GroupKFold(n_splits=iRun) # leave one out, initialize GroupKFold with Nrun-1 splits
    performance_this_fold = np.zeros(gkf.n_splits)
    for i_fold, (train_idx, test_idx) in enumerate(gkf.split(X=X, y=Y, groups=Gr)):
        # print("Indices of our test-samples: %r" % test_idx.tolist())
        # print("... which correspond to following runs: %r" % Gr[test_idx].tolist(), '\n')
        # Implement your ToDo here!
        X_test, X_train=X[test_idx], X[train_idx]
        Y_test, Y_train=Y[test_idx], Y[train_idx]
        
        pipe.fit(X_train, Y_train)
        preds = pipe.predict(X_test)
#         print('TRUE:',Y_test)
#         print('Pred:',preds)
#         print("Accuracy test: %.2f" % (preds == Y_test).mean())

#        performance = roc_auc_score(Y_test,  pipe.predict_proba(X_test), multi_class='ovr') # 'ovr'(One-vs-rest): Computes the AUC of each class against the rest, This treats the multiclass case in the same way as the multilabel case. Sensitive to class imbalance even when average == 'macro', because class imbalance affects the composition of each of the ‘rest’ groupings.
        # in the pairwise AUC calculation, if a sample is labeled wrongly as a another class (not in paired), it will be dropped from the calculation, resulting in a higher rate
        performance = roc_auc_score(Y_test,  pipe.predict_proba(X_test),  multi_class='ovo', average = 'macro') # 'ovo' (One-vs-one) Computes the average AUC of all possible pairwise combinations of classes. Insensitive to class imbalance when average == 'macro'.

        performance_this_fold[i_fold] = performance
        
    

    mean_performance_this_participant = performance_this_fold.mean()
    print(f'Performance for participant#{i_sub} over all {i_fold+1} folds: mean: {mean_performance_this_participant}, std: { performance_this_fold.std()}')
    print(f'   Performance of each fold: {performance_this_fold}')
    all_performance[i_sub] = mean_performance_this_participant

print('\nAverage performance mean (std): %.3f (%.3f)' % (all_performance.mean(), all_performance.std()))

Performance for participant#0 over all 5 folds: mean: 0.7218253968253968, std: 0.04981636724969173
   Performance of each fold: [0.64087302 0.69444444 0.73293651 0.78174603 0.75912698]
Performance for participant#1 over all 5 folds: mean: 0.7748263888888889, std: 0.06990644928212678
   Performance of each fold: [0.85069444 0.77083333 0.82118056 0.64670139 0.78472222]
Performance for participant#2 over all 5 folds: mean: 0.882638888888889, std: 0.027037666270899662
   Performance of each fold: [0.91840278 0.90625    0.84809028 0.8828125  0.85763889]
Performance for participant#3 over all 5 folds: mean: 0.8015625, std: 0.046802928132651335
   Performance of each fold: [0.81597222 0.71527778 0.84548611 0.79340278 0.83767361]
Performance for participant#4 over all 5 folds: mean: 0.8345486111111111, std: 0.029121161592030195
   Performance of each fold: [0.83767361 0.79427083 0.81857639 0.83940972 0.8828125 ]
Performance for participant#5 over all 8 folds: mean: 0.8332407407407407, std: 0.0